Сбор данных с помощью FlightRadarAPI

In [5]:
import math
import pandas as pd
import numpy as np
from FlightRadar24 import FlightRadar24API
fr_api = FlightRadar24API()

In [3]:
# Получаем координаты аэропорта Сочи
sochi_airport = fr_api.get_airport(code="URSS")
sochi_airport_coords = [sochi_airport.latitude, sochi_airport.longitude]
sochi_airport_coords

[43.449921, 39.956581]

In [3]:
# Функция для вычисления границ квадрата, центром которого является аэропорт Сочи
def compute_bounds(airport):
    arc_length = 111.3
    down_latitude = airport.latitude - 200/arc_length
    longitude_degree_km = arc_length * math.cos(math.radians(down_latitude))
    right_longitude = airport.longitude + 200/longitude_degree_km

    up_latitude = airport.latitude + 200/arc_length
    longitude_degree_km = arc_length * math.cos(math.radians(up_latitude))
    left_longitude = airport.longitude - 200/longitude_degree_km
    bounds = f"{up_latitude},{down_latitude},{left_longitude},{right_longitude}"
    return bounds

In [4]:
# Получаем полёты, которые обрабатываются прямо сейчас в этой области
bounds = compute_bounds(sochi_airport)
flights = fr_api.get_flights(
    bounds=bounds,
    details=True
)

In [5]:
# От полёта нужно:
# id, altitude, heading

d = {'id': [], 'altitude': [], 'heading': []}

df = pd.DataFrame(data=d)

In [6]:
# функция получения данных полёта и записи их в датафрейм
def update_data(fr_api, bounds, df):
    flights = fr_api.get_flights(
        bounds=bounds,
        details=True
    )
    for i in range(len(flights)):
        id = flights[i].id
        altitude = flights[i].altitude
        heading = flights[i].heading
        df.loc[len(df.index)] = [id, altitude, heading]

In [16]:
# вызываем функцию в цикле, крутим день
# import time
# i = 0
# while i != 2*60*24:
#    update_data(fr_api, bounds, df)
#    time.sleep(30)
#    i += 1

#df.head()

KeyboardInterrupt: 

In [17]:
# преобразуем в csv файл
#df.to_csv('data/1day_data.csv', index=False)

In [18]:
# проверяем, что вышло
df_r = pd.read_csv("data/1day_data.csv")
df_r

,id,altitude,heading
0,3265ef91,35000,91
1,3265cea8,32975,92
2,3265ac16,37000,91
3,32657ea3,35000,92
4,32662398,22325,18
...,...,...,...
26816,32695334,37050,98
26817,32699897,35000,92
26818,32691649,18650,66
26819,3269593e,33025,270
